# 04. Late Fusion and Evaluation
Combine image and tabular predictions

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
from fusion import compare_fusion_methods, generate_gradcam_samples
from image_model import ImageRegressionModel
import torch

print("\n📝 Fusion Strategy:")
print("  - Image & tabular models predict in log space")
print("  - Predictions are converted to actual prices: exp(log_pred)")
print("  - Fusion operates on actual price predictions")
print("  - All metrics evaluated on actual prices")

# Load validation data with predictions
val_img = pd.read_csv('../data/processed/val_with_image_preds.csv')
val_tab = pd.read_csv('../data/processed/val_with_tabular_preds.csv')

## Compare Fusion Methods

In [3]:
best_method, best_preds, results = compare_fusion_methods(
    val_img['image_prediction'].values,
    val_tab['tabular_prediction'].values,
    val_img['price'].values
)

# Display all results
print("\n" + "="*80)
print("DETAILED RESULTS")
print("="*80)
for method, result in results.items():
    print(f"\n{method.upper()}:")
    print(f"  RMSE: ${result['metrics']['rmse']:,.2f}")
    print(f"  MAE:  ${result['metrics']['mae']:,.2f}")
    print(f"  R²:   {result['metrics']['r2']:.4f}")
    print(f"  MAPE: {result['metrics']['mape']:.2f}%")
    if result['weights']:
        print(f"  Weights: {result['weights']}")


COMPARING FUSION METHODS

📊 Using 2422/2432 validation samples with both predictions

1. Simple Average (50/50):     RMSE=$139,618.56, R²=0.8489
2. Weighted Average (config):  RMSE=$112,884.26, R²=0.9012
   (Image: 0.4, Tabular: 0.6)
3. Adaptive Weighted:          RMSE=$25,324.89, R²=0.9950
   (Image: 0.00, Tabular: 1.00)
4. Max Fusion:                 RMSE=$170,341.98, R²=0.7751
5. Min Fusion:                 RMSE=$217,304.89, R²=0.6340

🏆 Best method: ADAPTIVE_WEIGHTED
   RMSE: $25,324.89
   R²: 0.9950

DETAILED RESULTS

SIMPLE_AVERAGE:
  RMSE: $139,618.56
  MAE:  $87,714.14
  R²:   0.8489
  MAPE: 18.53%
  Weights: {'image': 0.5, 'tabular': 0.5}

WEIGHTED_AVERAGE:
  RMSE: $112,884.26
  MAE:  $70,320.04
  R²:   0.9012
  MAPE: 14.87%
  Weights: {'image_model': 0.4, 'tabular_model': 0.6}

ADAPTIVE_WEIGHTED:
  RMSE: $25,324.89
  MAE:  $4,323.22
  R²:   0.9950
  MAPE: 0.66%
  Weights: {'image_weight': np.float64(0.0), 'tabular_weight': np.float64(1.0)}

MAX_FUSION:
  RMSE: $170,341.98
  

## Generate Grad-CAM Visualizations

In [ ]:
model = ImageRegressionModel()
checkpoint = torch.load('../models/best_image_model.pth', weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])

generate_gradcam_samples(model, val_img, n_samples=10)

✅ Complete! Check:
- data/predictions/final_predictions.csv
- reports/gradcam_samples/
- MLflow UI (run: mlflow ui)